In [1]:
pip install pyyaml pandas

  Using cached numpy-2.3.1-cp312-cp312-win_amd64.whl.metadata (60 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   ---------------------------------------- 0.0/11.0 MB ? eta -:--:--
   - -------------------------------------- 0.5/11.0 MB 3.4 MB/s eta 0:00:04
   -- ------------------------------------- 0.8/11.0 MB 1.7 MB/s eta 0:00:07
   --------- ------------------------------ 2.6/11.0 MB 4.2 MB/s eta 0:00:02
   --------------- ------------------------ 4.2/11.0 MB 5.1 MB/s eta 0:00:02
   -------------------- ------------------- 5.8/11.0 MB 5.7 MB/s eta 0:00:01
   -------------------------- ------------- 7.3/11.0 MB 6.0 MB/s eta 0:00:01
   ------------------------------- -------- 8.7/11.0 MB 6.2 MB/s eta 0:00:01
   ------------------------------------- -- 10.2/11.0 MB 6.3 MB/s eta 0:00:01
   -------------------------------------


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import yaml
import pandas as pd
from collections import defaultdict

# Folder where YAML data is stored
base_path = r"C:\Users\Heram Ramesh\OneDrive\Desktop\stock\dataset"  
# Create a dictionary to store data per symbol
stock_data = defaultdict(list)

# Loop through month folders
for month_folder in os.listdir(base_path):
    month_path = os.path.join(base_path, month_folder)

    if os.path.isdir(month_path):
        for file_name in os.listdir(month_path):
            if file_name.endswith(".yaml"):
                file_path = os.path.join(month_path, file_name)

                with open(file_path, 'r') as file:
                    day_data = yaml.safe_load(file)

                    # ✅ Correct for YAML files that are lists of dictionaries:
                    for record in day_data:
                        symbol = record.get("Ticker")
                        if symbol:
                            stock_data[symbol].append(record)


# Save each symbol's data to a CSV
output_dir = "stock_csvs"
os.makedirs(output_dir, exist_ok=True)

for symbol, records in stock_data.items():
    df = pd.DataFrame(records)
    df.sort_values('date', inplace=True)
    df.to_csv(os.path.join(output_dir, f"{symbol}.csv"), index=False)

print("✅ CSV export complete.")


✅ CSV export complete.


In [3]:
import os
import pandas as pd

csv_dir = "stock_csvs"  # after you move the CSVs
all_data = []

for file in os.listdir(csv_dir):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(csv_dir, file))
        df['symbol'] = file.replace(".csv", "")  # Add symbol column
        all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)
combined_df.to_csv("combined_dataset.csv", index=False)
print("✅ Combined CSV saved.")


✅ Combined CSV saved.


In [5]:
import pandas as pd

df = pd.read_csv("combined_dataset.csv")

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Ensure numeric types
numeric_cols = ['open', 'high', 'low', 'close', 'volume']
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce')

# Drop rows with missing data (optional)
df.dropna(subset=['open', 'high', 'low', 'close', 'volume'], inplace=True)

# Sort by date and symbol
df.sort_values(by=['symbol', 'date'], inplace=True)

df.to_csv("cleaned_dataset.csv", index=False)
print("✅ Data cleaned and saved.")


✅ Data cleaned and saved.


In [4]:
import pandas as pd

df = pd.read_csv("combined_dataset.csv")

# Count missing values in each column
print(df.isnull().sum())


Ticker    0
close     0
date      0
high      0
low       0
month     0
open      0
volume    0
symbol    0
dtype: int64


In [7]:
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    port=3306,
    user="root",
    password="Ram131203",
)

cursor = conn.cursor()
cursor.execute("CREATE DATABASE stock_data;")
#print("✅ Database created.")

conn.close()


In [13]:
#connect MySql
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    port=3306,
    user="root",
    password="Ram131203",
    database="stock_data"
)

cursor = conn.cursor()



In [9]:
cursor.execute("""
CREATE TABLE stock_prices (
    id INT AUTO_INCREMENT PRIMARY KEY,
    symbol VARCHAR(10),
    date DATE,
    open FLOAT,
    high FLOAT,
    low FLOAT,
    close FLOAT,
    volume BIGINT
);
""")
conn.commit()
print("✅ Table 'stock_prices' created.")


✅ Table 'stock_prices' created.


In [10]:
import pandas as pd

# Load cleaned data
df = pd.read_csv("cleaned_dataset.csv")

# Convert date column to datetime if not already
df['date'] = pd.to_datetime(df['date'])

# Insert data row by row
for _, row in df.iterrows():
    cursor.execute("""
        INSERT INTO stock_prices (symbol, date, open, high, low, close, volume)
        VALUES (%s, %s, %s, %s, %s, %s, %s)
    """, (
        row['symbol'],
        row['date'].date(),
        row['open'],
        row['high'],
        row['low'],
        row['close'],
        int(row['volume'])
    ))

conn.commit()
print("✅ All rows inserted into stock_prices table.")


✅ All rows inserted into stock_prices table.


In [14]:
cursor.execute("SELECT * FROM stock_prices LIMIT 5;")
for row in cursor.fetchall():
    print(row)


(1, 'ADANIENT', datetime.date(2023, 10, 3), 2418.0, 2424.9, 2372.0, 2387.25, 2019899)
(2, 'ADANIENT', datetime.date(2023, 10, 4), 2402.2, 2502.75, 2392.25, 2464.95, 2857377)
(3, 'ADANIENT', datetime.date(2023, 10, 5), 2477.95, 2486.5, 2446.4, 2466.35, 1132455)
(4, 'ADANIENT', datetime.date(2023, 10, 6), 2466.35, 2514.95, 2466.05, 2478.1, 1510035)
(5, 'ADANIENT', datetime.date(2023, 10, 9), 2440.0, 2459.7, 2411.3, 2442.6, 1408224)


In [15]:
pip install streamlit mysql-connector-python python-dotenv pandas numpy


Note: you may need to restart the kernel to use updated packages.


In [17]:
!pip install python-dotenv



In [18]:
from dotenv import load_dotenv


In [20]:
pip install streamlit

Note: you may need to restart the kernel to use updated packages.
